In [1]:
import numpy as np
import gmaps
import gmaps.datasets
import os
import requests
from ipywidgets.embed import embed_minimal_html

## Statistics:
- Annual PM2.5 Average in India
- Annual PM2.5 Average in New Delhi
- Heat Map of PM2.5 Average in New Delhi for 2016
- Heat Map of PM2.5 Averages in India for 2016
- Most polluted and least polluted cities in India
- Most polluted and least polluted neighborhoods in New Delhi

In [2]:
f = open('id-mappings/city-ids.txt', 'r')
cities = f.readlines()
cities = map(lambda elem: elem.split(","), cities)[1:]
cities = map(lambda elem: [elem[0], elem[1], elem[2], elem[3][:-1]], cities)
cities = filter(lambda elem: elem[2].isdigit(), cities)

In [3]:
india_data = {}
for elem in cities:
    _, stateName, _, cityName = elem
    f = open('data/{}_{}.txt'.format(stateName, cityName), 'r')
    city_data = f.readlines()
    city_data = filter(lambda elem: elem != "\n", city_data)
    city_data = map(lambda elem: elem.split(","), city_data)
    city_data = filter(lambda elem: elem[0] == "2016", city_data)
    city_data = map(lambda elem: float(elem[1].rstrip("\n")), city_data)
    if len(city_data) > 0:
        india_data[(stateName, cityName)] = np.mean(city_data)


In [4]:
india_data = [[k, v] for (k, v) in india_data.iteritems()]
india_data = sorted(india_data, key=lambda x: x[1])

In [5]:
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [6]:
def decode_address_to_coordinates(address):
        params = {
                'address' : address,
                'sensor' : 'false',
        }  
        url = 'http://maps.google.com/maps/api/geocode/json'
        r = requests.get(url, params = params)
        return r.json()['results'][0]['geometry']['location']

In [7]:
print decode_address_to_coordinates("New Delhi, Delhi")

{u'lat': 28.6139391, u'lng': 77.2090212}


In [8]:
locations = []
for (state, city), val in india_data:
    locations.append([(state, city), decode_address_to_coordinates("{}, {}".format(city, state)).values()])

In [9]:
india_coordinates = decode_address_to_coordinates("India").values()
fig = gmaps.figure(center=india_coordinates, zoom_level=4)

weights = map(lambda x: x[1], india_data)
coordinates = map(lambda x: x[1], locations)
heatmap_layer = gmaps.heatmap_layer(coordinates, weights=weights)
heatmap_layer.max_intensity = 200
heatmap_layer.point_radius = 2.0
heatmap_layer.dissipating = False
fig.add_layer(heatmap_layer)

info_box_template = """
<div>
<p><b>City:</b> {0}, {1}</p>
<p><b>PM2.5:</b> {2:.2f}</p>
</div>
"""
city_info = [info_box_template.format(city_data[0][1], city_data[0][0], city_data[1]) for city_data in india_data]

marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig.add_layer(marker_layer)

fig

A Jupyter Widget

In [10]:
embed_minimal_html('national-aq.html', views=[fig])